### 15.066 Optimization Final Project Proposal (July 2021)
Lauren Heintz, Gabriel Pascualy, Logan Greene, Lisa Schleuter, Jomi Kramer, Nick Esposito  
LGO 2023  
August 14, 2021

<h1><center>OFF GRID HOUSING ENERGY</center></h1> 

## 1. Introduction

As the effects of global warming become increasingly apparent, people are looking towards alternative energy and ways to live off the grid. Living off the grid means that you do not rely on public utilities for utilities like electricity, running water, sewer, natural gas, or heat, but rather you use independent sources. Because it is expensive to develop your own system to live off the grid, people take a hybrid approach to alternative energy, meaning that they utilize small capacity renewable energy products while still utilizing minimal public utilities.    

For many Americans, the high capital costs of alternative energy products are an obstacle to transition to low-carbon electricity. The amortization of the initial investment for off-grid energy must be cheaper than the cost of continuing to pay for grid-energy.  Off-grid energy has become more popular recently due to increasing ease-of-use for non-commercial use, and an increased awareness of energy consumption and longevity of resources.  Over time, the financial benefit of renewable energy products can save a substantial amount of money on utilities while minimizing the impact on the environment. Just a decade ago, it was much cheaper to build a new coal-burning power plant than it was to build a large capacity wind or solar plant. Since then, the price per MWh has declined by 89% for large capacity renewable energy sources which has trickled down into the smaller capacity renewable energy product market (Roser, 2020)1. Recently, reductions in the initial cost of renewable energy products have been spurred on by improvements in manufacturing processes and advances in technology which have made living off the grid more accessible to a wider range of income levels. As of 2017, approximately 180,000 families live off the grid which is expected to significantly increase every year (Infoplease, 2020)2.  

Our project aims to assist families considering living off the grid. Our goal is to develop a tool that will help evaluate how to minimize the overall cost associated with off-grid products based on the energy resources available, regional efficiencies of wind and solar energy, and costs of various energy products. The tool will accept inputs for average electricity demand, regional information associated with production of wind and solar energy, initial and monthly budget constraints, and costs associated with all energy products. Based on these parameters, the user will be able to determine the optimal mix of energy products (including utilizing the grid) to meet demand while minimizing financial costs.   

## 2. Background
####  Demand
This optimization problem starts with energy demand data. We chose to use hourly electricity demand data over the course of the year in units of kwh (kilowatt-hours). This data was procured from << EIA - US Energy Information Administration (4) >> which gives total grid energy consumption for various regions of the United States. Using information on the percent breakdown of residential energy usage from the grid, and census data on the population of the region, we derived an average hourly energy consumption metric per household, per hour, for every hour of a standard 365 day calendar year, using the readily available data for year 2020 (removing February 29th since 2020 is a leap year). This estimation method aligned closely with the total estimated energy consumption per household from << EIA - US Energy Information Administration (5)>> . 

  
<< we can insert an equation here to represent the math if we want >> 


Then, the average hourly electricity demand for each day was utilized in the model to capture the changes in demand during different seasons and for different parts of the country with different weather patterns. The user interface in this notebook allows the user to choose from a few predefined locations in a drop-down list which capture demand data from different regions of the United States. Users can also procure and upload their own demand data in the format required for the model.  

  

Additionally, we provide a feature which allows the user to choose the times of year during which the home would be used. For a full-time resident, this would be the full year. However, if this off-grid house is a summer vacation home or a winter vacation home, that may change the energy demand needs. As a result, the recommended product suggestions output from our model will change according to this data. 
  

<< insert some sample graphs/charts of demand data from somewhere showing a seasonal pattern >> 

  
A complementary set of data required to supplement this energy demand data is data describing the efficiency of different energy generation products based on geographical location. The efficiency of solar and wind can vary greatly based on the time of year, the amount of sunlight a region receives, temperature, and local weather patterns for that area. We acquired data on the efficiency of solar power and wind power from <<<  Matasci, S (6)> which allows an input of any city, and in return provides hourly efficiency data. This data is used in the optimization to understand the optimal mix of energy products to support the user’s home, depending on the location.  


<< insert visuals on efficiency data, if we want to >> 

#### Supply 
In the optimization model, the supply of energy to meet the demand comes from several possible power sources. These can first be divided by on-grid and off-grid power sources. Within off-grid power sources, we have energy generation products and energy storage products. The two types of energy generation products are wind and solar power. The table below defines the mix of products we chose to input into our model, so the output of the model provides an optimal combination of these power sources that meets energy demand and other constraints, while minimizing cost to the resident.       
  

<center> Available Products </center>

Product |Generation/Storage |Energy Type |On-Grid/Off-Grid
-----|-----|-----|----| 
**Grid Power**|Generation|Grid|On-Grid
**Solar S**|Generation|Solar|Off-Grid
**Solar L**|Generation|Solar|Off-Grid  
**Wind Turbine S**|Generation|Wind|Off-Grid
**Wind Turbine L**|Generation|Wind|Off-Grid
**Battery**|Storage|Storage|Off-Grid

<center> Table 1 </center>

Each of these products is defined with a maximum amount of energy output in kwh (kilowatt-hours). These detailed specifications are defined in a configuration CSV file which can be uploaded by the user for custom products they would like to consider. Each power generation product (solar and wind) has a maximum capacity in kwh, which is multipled by an efficiency metric mentioned in the demand section above. This provides the expected hourly power generation in kwh of each product over every hour of the year. 


The energy generation products and storage products interact directly with each other. In our case, "Storage" products are batteries and can be used to store excess energy generated, or they can be used to meet the energy demand of the customer. Our system is set up so that excess energy generated daily from wind and solar products can be stored indefinitely and used for other days when demand is higher. We operate under the assumptions that you would not pull energy from the grid and store it on your batteries, and that the intention of using battery energy is to minimize the amount of on-grid energy consumed. Of practical note is that net-metering is a popular option for households that produce alternative energy and do not want to invest in batteries. Excess energy is contributed back to the grid, and the customer builds an “energy credit” for future use. Net-metering is different from selling energy to the grid which, depending on the state, could be less profitable.  


<< graph of line plots showing: demand, energy generated, energy stored, and grid power over the same x-axis of time >> 

  

Grid paragraph here: you can select if you're connected to the grid. if you are, you can sell energy back to the grid at a discounted rate.  


<< could put a visual here showing selling energy back to the grid >> 

  

A household's ability to satisfy demand requirements is a combination of the energy generation products and the storage products. The output of our tool suggests a cost-optimal number of products from the list given in Table 2 & 3.    


<center> Sample Output </center>

Product |Units to Purchase|
-----|-----|
**Solar S**|1|
**Solar L**|1|  
**Wind Turbine S**|1|
**Wind Turbine L**|1|
**Battery**|1|

<center> Table 2 </center>

Grid Usage |Kilowatt-Hours|
-----|-----|
**Grid Energy Consumed**|1|
**Energy Sold to Grid**|1|

<center> Table 3 </center>


#### Cost

<u>Off Grid Product Cost Structure</u>  

Each product from the product list has a set of cost variables associated with it that are defined below:
- **"Opening Cost"**: The baseline set up cost to install the wiring and necessary equipment to connect an external energy source (solar, wind, batteries) to a house in a way that can be used by the household. This would only be done once for each type of energy source.
- **"Unit Cost"**: The cost per unit of a product, such as a solar panel or a wind turbine, that is incurred once per unit at the time of initial purchase.
- **"Maintenance Cost"**: The estimated annual cost to upkeep and maintain a given product from the product list.

<u>Grid Connection Cost Structure</u> 
- **"Opening Cost"**: The baseline set up cost to install the wiring and necessary equipment to connect the house the nearest grid power source, assuming that it is not yet connected.
- **"Grid Hourly Cost"**: The cost per kilowatt-hour to pull energy directly from the grid and use it in a home.
- **"Grid Hourly Revenue"**: The price per kilowatt-hour the excess energy can be sold back to the grid.

## 3. Theoretical Formulation

### A. Demand
Let $h$ := be the hour of the year where total hours in a year = (24*365), $h \in H$ where $H = \{{0, 1, 2, ... 8760}\}$

Let $d_{h}$:= be the customer electricity demand in KwH during hour $h$ of the year where $d_{h} \geq 0$

### B. Power Source Parameters
##### Product Definition Parameters
Let $j$ := be a product $j$ $\in$ $J$ where $J = \{{"SmallSolar", "LargeSolar", "SmallWind", "LargeWind", "battery", "grid"}\}$

Let $E$ := be a set containing indices of products which generate electricity except for "grid"

Let $S$ := be a set containing indices of products which store electricity 

Let $et$ := be an energy type where $et \in T = \{{"solar", "wind", "storage"}\}$

Let $ET_{et}$ := be a set containing all products of energy type $et$

##### Cost Paramaters
Let $oc_{j}$ := be the opening cost in US dollars of installing required infrastructure for product $j$ which belongs to an energy type $et$

Let $uc_j$ := be the unit cost in US dollars of purchasing an additional unit of a specific product $j$ where $uc_j \geq 0$

Let $mc_j$ := be the cost in US dollars of maintaining one unit of product $j$ where $mc_j \geq 0$

Let $Gc_h$ := be the cost of on-grid power in units of dollers per kilowatt hour for hour $h$  

Let $Gr$ := be the revenue rate in dollars per KwH that excess energy can be sold back to the grid. This number represents positive revenue.

##### Energy Conversion & Storage Parameters
Let $\rho_{h,j}$ := be the efficiency of product $j$ during hour $h$ of the year where $\rho_{h,j}$ is continuous from 0 to 1

Let $c_{j}$ := be the capacity of product $j$ in KwH if it generates or stores electricity


### C. Decision Variables
##### Product Decision Variables
Let $x_j$ := be a binary variable representing whether or not the opening cost of $j$'s energy type is paid for where $x_j \in$ {0,1}

Let $y_{j}$ := be an integer variable representing the number of units of product $j$ where $y_j \geq$ 0

##### Grid Power Decision Variables
Let $g_{h}$ := be the continuous variable that represents the amount of power needed from the grid in kilowatt-hours during hour $h$ of the year.

Let $ge_{h}$ := be the continuous variable that represents the amount of excess energy sold back to the grid in kilowatt-hours during hour $h$ of the year.

##### Storage Decision Variables
Let $es_{h,j}$ := be the continuous variable that represents the amount of energy stored by product $j$ (note: not all products can store energy)

Let $sc_{h,j}$ := be the continuous variable that represents the amount of stored consumed from product $j$ (note: not all products can store energy)

### D. Calculated Variables
##### Calculated Storage Variables
Let $ES_{h}$ := be the total energy stored this year through hour $h$ calculated by summing cumulative the energy stored each hour for all products $\sum_{j \in J} es_{h,j}$

Let $SC_{h}$ := be the total storage consumed this year through hour $h$ calculated by summing the cumulative energy consumed each hour for all products $\sum_{j \in J} sc_{h,j}$

Let $TT$ := be the total energy storage of the system calculated by summing the capacity of all products $\sum_{j \in J} c_{j}y_{j}$



### E. Objective Function
We want to minimize the cost; therefore, the objective function is the sum of various costs:

    a) opening cost (oc) the cost of initially setting up a new service

    b) unit cost (uc)
    
    c) maintenance cost (mc)

    d) cost of grid/hr (Gc) * grid hours consumed (g)

after subtracting the revenue from excess energy sold back to the grid:
    
    e) grid sell back rate (Gr) * energy sold to grid (ge)

$$\min \sum_{j \in J} oc_{j}x_j + \sum_{j \in J} uc_jy_j + \sum_{j \in J} mc_{j}y_{j} + \sum_{h \in H} {(Gc_h)(g_{h}) - (Gr)(ge_h)} $$

### F. Constraints

##### Demand Constraints
1) Meet customer demand for electricity per hour per month by summing the energy generation products ($E$) output, the stored energy ($S$) available, and grid power ($g$) available

$$ \forall h \in H:  \sum_{j\in E} \rho_{h,j} y_{j} + \sum_{j\in S}sc_{h,j} + g_{h} \geq d_{h}$$

##### Budget Constraints
2) You have an initial budget of the amount of capital you have on hand to shell out for initial set up.
$$  \sum_{j \in J} oc_{j}x_j + \sum_{j \in J} uc_jy_j \leq $200,000 $$

3) You also have a monthly budget.
$$ \sum_{j \in J} mc_{j}y_{j} \leq $500 $$

##### Set up and Installation Constraints
4) Ensure at least 1 $x_j$ per energy type is 1 when any $y_j$ of the energy type is $\geq 1$

$$ \forall j \in ET_{solar}: \sum_{j' \in ET_{solar}} x_{j'} * M >= y_{j}$$

$$ \forall j \in ET_{wind}: \sum_{j' \in ET_{wind}} x_{j'} * M >= y_{j}$$

$$ \forall j \in ET_{storage}: \sum_{j' \in ET_{storage}} x_{j'} * M >= y_{j}$$

5) Ensure that at most 1 $x_j$ per energy type is equal to 1

$$ \forall et \in ET: \sum_{j\in ET_{et}}x_j \leq 1$$

##### Storage Constraints
6) Total storage consumed at hour $h$ of the year cannot exceed electricity stored

$$ \forall h \in H\space: SC_{h} \leq ES_{h} $$ 

7) Storage available at hour $h$ of the year cannot exceed total storage capacity

$$ \forall h \in H\space: ES_{h} - SC_{h}  \leq TT $$

8) Electricity stored at hour $h$ of the year cannot exceed the sum of electricity produced, storage consumed, and grid usage minus demand at hour $h$ of the year and energy sold back to the grid at hour $h$. 

$$ \forall h \in H:\space \sum_{j\in S} es_{h,j} \leq g_{h} - d_{h} - ge_h + \sum_{j\in E} \rho_{h,j} y_{j} + \sum_{j\in S} sc_{h,j} $$

##### Grid Constraints

9) No energy can be sold back to the grid unless the grid is connected/installed to the house.

$$ \forall h \in H:\space ge_h \leq x_{j=grid}*M $$

$$ x_{j=grid} * M >= \sum_{h \in H} g_h$$

## 4. Interface/Gui
- description of features & capability
- hosted on binder and accessible via github
- explanation of the visuals included in the output

## 5. Results
#### Sub Experiments
Our team explored the effect of different capital budgets on the optimal product mix for minimizing grid-usage (Table 4). Three budget options were considered: low-budget, mid-range, and high-end. In the specific case of our pilot site (Asheville, NC), the high-end option was the only one that allowed full off-grid living.  

The break-even point is the time at which the energy savings offset the initial capital investment. For the Asheville site, this was found to be 47 years. The initial capital budget and pay-off periods are the biggest factors that will determine a household’s available product mix. The breakeven points for the Asheville area were fairly high, due to inconsistent wind efficiency and high energy demand in the summer. However, note that most of the population will use alternative energy sources in combination with net-metering, which drastically reduces the pay-off period. 

We also analyzed the effects of seasonal home usage compared to full-year home usage. For the case of our pilot site, this showed both the relative costs of optimizing for various seasons, as well as the most economical mix, dependent on season (Table 5).  

Finally, we collected data for various regions in the U.S., and highlight the impact of the three extremes in Table 6 below. The differences in both demand and supply make this a complex problem.  This analysis does not consider any state-specific tax breaks, which could greatly influence the optimal product mix (10). Out of the three sites highlighted below, Asheville requires a variety of products and a high quantity of batteries to go off-grid, due to inconsistency in alternative energy efficiencies, as well as high energy demand in the summer. Sedona is a very cost-effective region to go off-grid, as energy sources are very consistent in this area. Yakima is a very windy region and is the site of many commercial wind farms. However, the high variability in wind efficiency requires a high number of batteries. Cost inputs can easily be adjusted in our interface to account for state tax breaks, and this product mix can be further customized. 

This tool can be used and scaled for several different use-cases.  Our focus for this tool was primarily for individual household users. Though our tool does not incorporate any policy-related factors such as tax incentives, this could easily be added. The sensitivities for different regions and demographics that are shown through this tool could be used by policy-makers to determine incentive thresholds to help their area reach certain alternative-energy goals. Additionally, several of our team-members plan to build partial or fully off-grid homes in the future, and we also incorporated feedback from people who currently live in partial or fully off-grid homes. This “customer research” helped us incorporate realistic factors and define our decision and static variables. Many classmates, friends, and family members have already reached out to request the use of our tool for evaluating alternative energy for their specific situation. 

- in discussion of results, use the size of the solar panels and space needed for wind turbines to output how much sqft or acreage is needed to support this amount of product 

    - could also be implemented as a constraint too but TBD for now 

## 6. Optimization Tool

In [2]:
from offgridoptimizer.interface import OffGridOptimizer
ogo = OffGridOptimizer()
# ogo.load_sheets(None)
# ogo.optimize(None)
ogo.interface

ModuleNotFoundError: No module named 'tabulate'

## 7. Discussion

## 8. Appendix - Assumptions